# Build Chatbot NLU
## 1. Load CSV Dataset

In [1]:
import pandas as pd
import os

Questions_PATH="./"

def load_dataset(dataset_path=Questions_PATH):
    csv_path = os.path.join(dataset_path, "question-intent.csv")
    return pd.read_csv(csv_path)

dataset = load_dataset()
#dataset

## 2. Preprocess Dataset - Input
2.1 Seplit Questions into Words

2.2 Remove Stopwords

2.3 Root words

2.4 Re-join word in one statement

In [3]:
from nltk import word_tokenize
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
from string import punctuation

dataset['جذور السؤال'] = ''

st = ISRIStemmer()
_stopwords = set(stopwords.words('arabic') + list(punctuation))

def preprocess(text):
    words = word_tokenize(str(text))
    words = [word for word in words if word not in _stopwords]
    words = [st.stem(word) for word in words]
    return ' '.join(words)

for index, row in dataset.iterrows():
    dataset['جذور السؤال'][index] = preprocess(row['السؤال'])
    
#dataset

## 3. Preprocess Dataset - Output

In [4]:
from sklearn.preprocessing import OneHotEncoder

label_cat = dataset[['الخدمة']]

cat_encoder = OneHotEncoder()
label_cat_1hot = cat_encoder.fit_transform(label_cat).toarray()
#label_cat_1hot

## 4. Shuffle and Split into train, and test sets

In [ ]:
import numpy as np

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

sentences = dataset['جذور السؤال'].to_numpy()
X, Y = unison_shuffled_copies(sentences, label_cat_1hot)

training_size = int(len(X) * 0.8)
train_X, test_X = X[:training_size], X[training_size:]
train_Y, test_Y = Y[:training_size], Y[training_size:]

print(len(train_X))
print(len(train_Y))

print(len(test_X))
print(len(test_Y))

## 5. Convert to Sequence, and Padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

oov_tok = "<OOV>"
trunc_type='post'
padding_type='post'
max_length = 100
vocab_size = 1000
embedding_dim = 16

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(X)

word_index = tokenizer.word_index

train_X_seq = tokenizer.texts_to_sequences(train_X)
train_X_pad = pad_sequences(train_X_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_X_seq = tokenizer.texts_to_sequences(test_X)
test_X_pad = pad_sequences(test_X_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

## 6. Build Deep Learning Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adamax

lstm_dim = 32
dense_dim = 16

opt = Adamax(learning_rate=0.03, beta_1=0.8, beta_2=0.9999)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#model.summary()

## 7. Train, and Validate the Model

In [ ]:
num_epochs = 40
history = model.fit(train_X_pad, train_Y, epochs=num_epochs, validation_data=(test_X_pad, test_Y))

## 8. Plot Accuracy and Loss / Epochs

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

## 9. Test and Try

In [ ]:
sen = preprocess('قم بتجديد رخصتي التجارية')
train_X_seq = tokenizer.texts_to_sequences([sen])
train_X_pad = pad_sequences(train_X_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

result = model.predict(train_X_pad)
print(cat_encoder.categories_[0][result.argmax()])
print(result.max())

In [ ]:
nr = result.argsort()
predicted_result = nr[0][-4:]
for i in predicted_result[::-1]:
    print(result[0][i])
    print(cat_encoder.categories_[0][i])